In [1]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
# data_as_key = ("ids_to_validate", "data", "data")

bert_data_key = ("ids_to_validate", "data", "bert_tags")
# tags_data = client.get(bert_data_key)[-1]
# labels_for_train = tags_data["gutenTag"]
# print(len(labels_for_train))

In [2]:
import pandas as pd
import bson
import pymongo
from tqdm import tqdm
tqdm.pandas()
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

/home/ubuntu/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
from fastai import *
from fastai.vision import *

In [4]:
from tagger import config, train_model, stringConverter
from tagger import predict

/home/ubuntu/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: thinc.extra.search.Beam size changed, may indicate binary incompatibility. Expected 112 from C header, got 120 from PyObject
  return f(*args, **kwds)


In [5]:
import transformers

In [6]:
transformers.__version__

'2.5.1'

In [7]:
argsTags = {
    "labels": client.get(bert_data_key)[-1]["gutenTag"],
    "model": load_learner("models", "guten_tag.pkl"),
}

In [8]:
def changeTags(bson_id, new_tags):
     
    info = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
#     if "bert_tags" in info:
#         print("BEFORE:", info["bert_tags"])
        
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                               {"$set":{"bert_tags": new_tags}})

    info = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
#     print("AFTER:", info["bert_tags"])

In [9]:
db = db_client["tweets_pipeline_v2"].find({"$or":[
                                                    {"status": "pipelined"},                                                                                                                        
#                                                     {"status": "reserved"},                                                       
#                                                       {"status": "deleted"}, 
#                                                       {"status": "deleted_from_analytics"}
                                                 ],
                                                   "extracted_text": {"$exists": True},                                               
#                                                       "GutenTagTrash": {"$exists": False},

#                                                'tweet_timestamp': {'$gte': datetime(2020,2,4)} #algoStarts                                              
                                          }, no_cursor_timeout=True)



for i in tqdm(db, total = db.count()):
    _id = str(i["_id"])
    text = i["extracted_text"]
    if len(text) > 0:
        tagsPredict = predict(stringConverter(i["extracted_text"], use_mister_proper=True), argsTags)
        changeTags(_id, tagsPredict)
        
db.close()        

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  from ipykernel import kernelapp as app
9503it [22:12,  7.13it/s]                           


In [12]:
df = pd.read_csv("data-1596200464136.csv")
df

,ticker,name,cik,legal_name,short_description,long_description
0,EPR,EPR Properties,1045450,EPR PROPERTIES,EPR Properties is a real estate investment tru...,EPR Properties is a real estate investment tru...
1,SAVE,Spirit Airlines Inc,1498710,"Spirit Airlines, Inc.","Spirit Airlines, Inc. is an ultra low-cost, lo...","Spirit Airlines, Inc. provides low-fare airlin..."
2,CPTA,Capitala Finance Corp,1571329,Capitala Finance Corp.,NaN,NaN
3,OII,"Oceaneering International, Inc.",73756,OCEANEERING INTERNATIONAL INC,"Oceaneering International, Inc. provides engin...","Oceaneering International, Inc. provides engin..."
4,CVI,CVR Energy Inc,1376139,CVR ENERGY INC,"CVR Energy, Inc. engages in the provision of e...","CVR Energy, Inc., through its subsidiaries, en..."
...,...,...,...,...,...,...
5847,ADNT,Adient plc,1670541,Adient plc,Adient plc is an automotive seating supplier t...,"Adient plc designs, manufactures, and markets ..."
5848,INSW,International Seaways Inc,1679049,"International Seaways, Inc.","International Seaways, Inc. operates a fleet o...","International Seaways, Inc. owns and operates ..."
5849,WSRC,Western Sierra Resource Corp,42050,WESTERN SIERRA MINING CORP,Western Sierra Resource Corporation engages in...,Western Sierra Resource Corporation engages in...
5850,ATRX,Adhera Therapeutics Inc,737207,"Marina Biotech, Inc.","Adhera Therapeutics, Inc., a biopharmaceutical...","Adhera Therapeutics, Inc., a biopharmaceutical..."


In [13]:
def tags_pred(x):
    if type(x) == str:
        return predict(stringConverter(x, use_mister_proper=True), argsTags)

In [14]:
df["categories"] = df["long_description"].progress_apply(lambda x: tags_pred(x))

100%|██████████| 5852/5852 [05:36<00:00, 17.38it/s] 


In [15]:
import pickle
pickle.dump( df, open( "comps-categories.p", "wb" ) )

In [41]:
df.to_excel("comps-categories.xlsx")

In [10]:
db_client["tweets_pipeline_v2"].find({
                                    "status": "graphicone_feed",
                                    "extracted_text": {"$exists": True},                                               
                                      }, no_cursor_timeout=True).count()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


2529